# 🚀 YOLOv12-Face Enhanced - Training Simple

Workflow : Installation → Configuration → Entraînement Enhanced → Résultats

## 1. 📦 Installation des dépendances

In [ ]:
# Installer les dépendances nécessaires
!pip install ultralytics gdown opencv-python -q

# Vérifier l'installation
import torch
print(f"✅ PyTorch: {torch.__version__}")
print(f"✅ CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")

## 2. 🔧 Configuration des fichiers Enhanced

In [ ]:
import os
import shutil
from pathlib import Path

# Gestion du répertoire de travail (compatible cloud)
try:
    work_dir = Path.cwd()
except (FileNotFoundError, OSError):
    # Fallback pour environnements cloud
    work_dir = Path('/content') if Path('/content').exists() else Path.home()
    os.chdir(work_dir)
    print(f"⚠️ Répertoire changé vers: {work_dir}")

print(f"📁 Répertoire de travail: {work_dir}")

# Vérifier si nous avons les configs Enhanced localement
configs_dir = work_dir / "scripts" / "configs"
has_local_configs = configs_dir.exists() and len(list(configs_dir.rglob('*.yaml'))) > 0

if has_local_configs:
    print("✅ Configurations Enhanced trouvées localement")
    # Restaurer les fichiers
    !python scripts/restore_configs.py
else:
    print("⚠️ Pas de configurations Enhanced - utilisation des configs par défaut")
    print("💡 Vous pouvez utiliser le modèle de base: yolov12n.yaml")

## 3. 📥 Préparation du dataset WIDERFace

In [ ]:
# Vérifier si le dataset existe déjà
if Path('datasets/widerface/data.yaml').exists():
    print("✅ Dataset WIDERFace déjà présent")
    DATA_CONFIG = 'datasets/widerface/data.yaml'
else:
    print("📥 Dataset WIDERFace non trouvé")
    print("💡 Vous pouvez:")
    print("   1. Utiliser un dataset COCO existant")
    print("   2. Télécharger WIDERFace manuellement")
    
    # Utiliser COCO comme fallback
    DATA_CONFIG = 'coco.yaml'
    print(f"📊 Utilisation du dataset: {DATA_CONFIG}")

## 4. 🏋️ Configuration et Entraînement

In [ ]:
# Configuration de l'entraînement
EPOCHS = 50  # Réduit pour tester
BATCH_SIZE = 16
DEVICE = 0 if torch.cuda.is_available() else 'cpu'

# Choisir le modèle (Enhanced si disponible, sinon de base)
enhanced_model = 'ultralytics/cfg/models/v12/yolov12-face-enhanced.yaml'
base_model = 'yolov12n.yaml'

if Path(enhanced_model).exists():
    MODEL = enhanced_model
    MODEL_NAME = 'yolov12-face-enhanced'
    print("🧠 Utilisation du modèle Enhanced avec modules d'attention")
else:
    MODEL = base_model
    MODEL_NAME = 'yolov12-face-base'
    print("⚙️ Utilisation du modèle de base")

print(f"\n📊 Configuration:")
print(f"   • Modèle: {MODEL}")
print(f"   • Dataset: {DATA_CONFIG}")
print(f"   • Epochs: {EPOCHS}")
print(f"   • Batch: {BATCH_SIZE}")
print(f"   • Device: {DEVICE}")

## 5. 🚀 Lancement de l'entraînement

In [ ]:
# Créer le dossier de sortie
!mkdir -p runs/train

# Lancer l'entraînement
print("🚀 Démarrage de l'entraînement...")
!yolo detect train model={MODEL} data={DATA_CONFIG} epochs={EPOCHS} batch={BATCH_SIZE} device={DEVICE} imgsz=640 project=runs/train name={MODEL_NAME} exist_ok=True save=True plots=True

print("\n🎉 Entraînement terminé !")

## 6. 📊 Validation et Résultats

In [ ]:
# Chemin vers le meilleur modèle
BEST_MODEL = f'runs/train/{MODEL_NAME}/weights/best.pt'

if Path(BEST_MODEL).exists():
    print(f"📊 Validation du modèle {MODEL_NAME}...")
    !yolo detect val model={BEST_MODEL} data={DATA_CONFIG} device={DEVICE}
    
    print(f"\n✅ Modèle sauvegardé: {BEST_MODEL}")
else:
    print("❌ Modèle non trouvé")

## 7. 📦 Export du modèle

In [ ]:
# Export en différents formats
if Path(BEST_MODEL).exists():
    print("📦 Export du modèle...")
    
    # ONNX pour déploiement
    !yolo export model={BEST_MODEL} format=onnx simplify=True
    
    # TorchScript pour PyTorch
    !yolo export model={BEST_MODEL} format=torchscript
    
    print("\n✅ Export terminé !")
    print(f"📁 Fichiers disponibles dans: runs/train/{MODEL_NAME}/weights/")
else:
    print("❌ Pas de modèle à exporter")

## 8. 🖼️ Test de détection

In [ ]:
# Test sur image d'exemple
if Path(BEST_MODEL).exists():
    # Chercher une image de test
    test_images = []
    
    # Chercher dans le dataset
    if Path('datasets/widerface/images/val').exists():
        test_images = list(Path('datasets/widerface/images/val').glob('*.jpg'))[:3]
    
    if test_images:
        print(f"🖼️ Test sur {len(test_images)} images...")
        
        # Créer dossier de résultats
        !mkdir -p test_results
        
        # Tester sur les images
        for img in test_images:
            !yolo detect predict model={BEST_MODEL} source={str(img)} save=True project=test_results name=demo exist_ok=True conf=0.5
        
        print("\n✅ Résultats sauvegardés dans test_results/demo/")
    else:
        print("⚠️ Aucune image de test trouvée")
        print("💡 Vous pouvez tester avec: yolo predict model=best.pt source=your_image.jpg")
else:
    print("❌ Modèle non disponible pour les tests")

## 9. 📈 Visualisation des résultats

In [ ]:
# Afficher les graphiques d'entraînement
from IPython.display import Image, display

results_path = Path(f'runs/train/{MODEL_NAME}')

if results_path.exists():
    print(f"📊 Résultats de l'entraînement {MODEL_NAME}:\n")
    
    # Graphiques à afficher
    plots = ['results.png', 'confusion_matrix.png', 'F1_curve.png', 'PR_curve.png']
    
    for plot in plots:
        plot_path = results_path / plot
        if plot_path.exists():
            print(f"📈 {plot}:")
            try:
                display(Image(filename=str(plot_path), width=800))
            except:
                print(f"   Disponible: {plot_path}")
        else:
            print(f"⚠️ {plot} non trouvé")
    
    # Afficher les métriques finales
    results_csv = results_path / 'results.csv'
    if results_csv.exists():
        print("\n📊 Métriques finales:")
        import pandas as pd
        df = pd.read_csv(results_csv)
        if len(df) > 0:
            last_row = df.iloc[-1]
            print(f"   mAP@0.5: {last_row.get('metrics/mAP50(B)', 'N/A'):.4f}")
            print(f"   mAP@0.5-0.95: {last_row.get('metrics/mAP50-95(B)', 'N/A'):.4f}")
            print(f"   Precision: {last_row.get('metrics/precision(B)', 'N/A'):.4f}")
            print(f"   Recall: {last_row.get('metrics/recall(B)', 'N/A'):.4f}")
else:
    print("❌ Dossier de résultats non trouvé")

## 💡 Résumé et Commandes Utiles

In [ ]:
# Résumé final
print("🎉 ENTRAÎNEMENT TERMINÉ !")
print("=" * 40)

if Path(BEST_MODEL).exists():
    print(f"✅ Modèle: {MODEL_NAME}")
    print(f"📁 Poids: {BEST_MODEL}")
    print(f"📊 Résultats: runs/train/{MODEL_NAME}/")
    
    print("\n🔧 Commandes utiles:")
    print(f"# Test sur webcam:")
    print(f"yolo predict model={BEST_MODEL} source=0 show=True")
    
    print(f"\n# Test sur vidéo:")
    print(f"yolo predict model={BEST_MODEL} source=video.mp4 save=True")
    
    print(f"\n# Benchmark:")
    print(f"yolo benchmark model={BEST_MODEL}")
    
    if 'enhanced' in MODEL_NAME:
        print("\n🧠 Modèle Enhanced avec modules d'attention entraîné avec succès !")
    else:
        print("\n⚙️ Modèle de base entraîné. Pour Enhanced, ajoutez les configurations.")
else:
    print("❌ Entraînement échoué - vérifiez les logs ci-dessus")

print("\n🚀 Le modèle est prêt pour la production !")